## Develop summary table of all Sensors in LoRa Database

In [8]:
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone

pd.options.display.max_rows = 200

In [9]:
gtw_file = '~/gateways.tsv'
df = pd.read_csv(gtw_file, sep='\t', parse_dates=['ts'])
df.head()

,dev_id,ts,ts_day,ts_hour,counter,gateway,snr,rssi,data_rate
0,ldds75-a84041281182a87f,2021-09-22 00:00:50,2021-09-22,2021-09-22 00:00,79,packetbroker,9.50,-72,SF7BW125.0
1,erslite-marquam-56a5,2021-09-22 00:00:57,2021-09-22,2021-09-22 00:00,40,packetbroker,10.25,-31,SF7BW125.0
2,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,msb-admin-a840411eb4f84150,10.50,-30,SF7BW125.0
3,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,packetbroker,4.00,-107,SF7BW125.0
4,ersco2-msb-admin-26db,2021-09-22 00:01:04,2021-09-22,2021-09-22 00:00,19959,packetbroker,-6.20,-122,SF7BW125.0


In [10]:
tz_ak = timezone('US/Alaska')
ts_start = (datetime.now(tz_ak) - timedelta(days=1)).replace(tzinfo=None)
ts_start

datetime.datetime(2021, 10, 21, 18, 6, 3, 902053)

In [11]:
df1d = df.query('ts >= @ts_start')

In [12]:
dfs = df1d.groupby('dev_id').agg(
    {
        'data_rate': 'last',
        'counter': ['first', 'last']
    }
)
dfs.columns = ['data_rate', 'counter_first', 'counter_last']
dfs['total_rdg'] = dfs.counter_last - dfs.counter_first + 1
dfs.head()

,data_rate,counter_first,counter_last,total_rdg
dev_id,,,,
boat-lt2-a726,SF10BW125.0,2081,2335,255
boat-lt2-a728,SF10BW125.0,279,533,255
boat-lt2-a72b,SF10BW125.0,9283,9537,255
boat-lt2-a72d,SF10BW125.0,3554,3808,255
boat-lt2-a72f,SF10BW125.0,270,11,-258


In [13]:
df_rcvd = df1d[['dev_id', 'counter']].drop_duplicates()
df_rcvd = df_rcvd.groupby('dev_id').count()
df_rcvd.columns = ['rcvd']
df_rcvd.head()

,rcvd
dev_id,
boat-lt2-a726,251
boat-lt2-a728,255
boat-lt2-a72b,250
boat-lt2-a72d,242
boat-lt2-a72f,243


In [14]:
df_final = dfs.join(df_rcvd)
df_final['success_pct'] = df_final.rcvd / df_final.total_rdg
df_display = df_final[['data_rate', 'success_pct']]
df_display['data_rate'] = df_display.data_rate.str.strip('.0')
df_display.columns = ['Data Rate', 'Success %']
df_display.index.name = 'Dev ID'
s2 = df_display.style.applymap(lambda v: 'color:red;' if v < 0.9 else None, subset=['Success %'])
s2

<ipython-input-14-eb52af82b5bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_display['data_rate'] = df_display.data_rate.str.strip('.0')


,Data Rate,Success %
Dev ID,,
boat-lt2-a726,SF10BW125,0.984314
boat-lt2-a728,SF10BW125,1.000000
boat-lt2-a72b,SF10BW125,0.980392
boat-lt2-a72d,SF10BW125,0.949020
boat-lt2-a72f,SF10BW125,-0.941860
boat-lt2-a84041552182436a,SF10BW125,0.964706
boat-lt2-a840417f8182436e,SF10BW125,0.956863
boat-lt2-phoenix-swd,SF7BW125,0.964706
elsysco2-seed-a81758fffe046262,SF8BW5,1.000000
